In [12]:
import mhcflurry, seaborn, numpy, pandas, pickle, sklearn, collections, scipy, time, logging
import mhcflurry.data
import mhcflurry.imputation
import fancyimpute

import sklearn.metrics
import sklearn.cross_validation
from collections import defaultdict

In [18]:
min_peptides_to_consider_allele = 50
max_ic50 = 50000
data_dir = "../data/"

In [19]:
all_train_data = mhcflurry.data.load_allele_datasets(
    data_dir + "bdata.2009.mhci.public.1.txt",
    use_multiple_peptide_lengths=True)

In [20]:
alleles = [
    "HLA-A0201",
    # "HLA-A0301",
    # "HLA-A0203",
    # "HLA-A2602",
    # "HLA-A2603",
    # 'HLA-B7301',
]
#alleles = alleles[:1] + alleles[-1:]
#alleles = [allele for allele in all_train_data if len(all_train_data[allele].Y) >= min_peptides_to_consider_allele]

In [21]:
all_train_data[alleles[0]].weights.std()

0.36749263596306014

In [22]:
set(len(x) for x in all_train_data[alleles[0]].original_peptides)

{8, 9, 10, 11, 12, 13, 14, 15}

In [23]:
set(len(x) for x in all_train_data[alleles[0]].peptides)

{9}

In [24]:
#train_data = dict((allele, data)
#                  for (allele, data) in all_train_data.items()
#                  if len(data.Y) >= min_peptides_to_consider_allele)
train_data = dict((allele, all_train_data[allele]) for allele in alleles)
print("Training data: %d / %d alleles" % (len(train_data), len(all_train_data)))

#test_data = mhcflurry.data.load_allele_datasets("/Users/tim/sinai/git/mhcflurry/bdata.2013.mhci.public.blind.1.txt")


Training data: 1 / 106 alleles


In [25]:
def log_to_ic50(log_value):
        """
        Convert neural network output to IC50 values between 0.0 and
        self.max_ic50 (typically 5000, 20000 or 50000)
        """
        return max_ic50 ** (1.0 - log_value)

def make_scores(y, y_pred, weights=None, sample_weight=None, threshold_nm=500):
    ic50_y = log_to_ic50(y)
    ic50_y_pred = log_to_ic50(y_pred) 
    return dict(
        auc=sklearn.metrics.roc_auc_score(ic50_y <= threshold_nm, y_pred, sample_weight=sample_weight),
        f1=sklearn.metrics.f1_score(ic50_y <= threshold_nm, ic50_y_pred <= threshold_nm, sample_weight=sample_weight),
        tau=scipy.stats.kendalltau(y_pred, y)[0],
    )    

def mean_with_std(grouped_column, decimals=3):
    pattern = "%%0.%df" % decimals
    return pandas.Series([
        (pattern + " +/ " + pattern) % (m, s) if not pandas.isnull(s) else pattern % m
        for (m, s) in zip(grouped_column.mean(), grouped_column.std())
    ], index = grouped_column.mean().index)

def allele_data_to_df(data):
    d = data._asdict()
    d["X_index"] = [x for x in d["X_index"]]
    d["X_binary"] = [x for x in d["X_binary"]]
    df = pandas.DataFrame(d).set_index('peptides')
    return df

def make_2d_array(thing):
    return numpy.array([list(x) for x in thing])

def df_to_allele_data(df):
    d = dict((col, df[col].values) for col in df)
    d["X_index"] = make_2d_array(d["X_index"])
    (d["max_ic50"],) = list(df.max_ic50.unique())
    return mhcflurry.data.AlleleData(peptides = df.index.values, **d)

def get_shuffled_fields(allele_data):
    """
    Parameters
    ----------
    allele_data : mhcflurry.data.AlleleData
    
    Returns shuffled array fields
    """
    original_peptides = data.original_peptides
    X = data.X_index 
    Y = data.Y
    weights = data.weights
    
    # shuffle all the  samples!
    shuffle_indices = np.arange(len(X))
    np.random.shuffle(shuffle_indices)
    
    X = X[shuffle_indices]
    Y = Y[shuffle_indices]
    weights = weights[shuffle_indices]
    original_peptides = original_peptides[shuffle_indices]
    
    return X, Y, weights, original_peptides

def collapse_9mer_affinities(Y_9mer_true, Y_9mer_pred, original_peptides):
    """
    Parameters
    ----------
    Y_9mer_true : np.array of float
        True regression target values for 9mers extracted from longer/shorter peptides
    
    Y_9mer_pred : np.array of float
        Predicted values for 9mers
    
    original_peptides : np.array of str
        Original peptides of varying length that 9mers were extracted from.
    """
    # collapse multiple 9mer predictions and measured values into 
    # smaller set of predictions for peptides of varying lengths
    peptide_to_true_affinity_dict = defaultdict(list)
    peptide_to_predicted_affinity_dict = defaultdict(list)
    for i, p in enumerate(original_peptides):
        peptide_to_true_affinity_dict[p].append(Y_9mer_true[i])
        peptide_to_predicted_affinity_dict[p].append(Y_9mer_pred[i])

    unique_peptides = list(sorted(set(peptide_to_predicted_affinity_dict.keys())))
    print("-- # unique peptides = %d" % (len(unique_peptides),))
    Y_true = np.array([
            np.mean(peptide_to_true_affinity_dict[p]) for p in unique_peptides ])
    Y_pred = np.array([
            np.mean(peptide_to_predicted_affinity_dict[p]) for p in unique_peptides])
    return Y_true, Y_pred

In [ ]:
dropout_probabilities = [0.0, 0.1, 0.5]

embedding_output_dims = [16, 32, 64, 128]
#embedding_output_dims = [4, 32]

#layer_sizes = [[4], [8], [16], [64], [128]]
layer_sizes_list = [[16], [64], [100], [128]]

activations = ["tanh"]

models_params_list = []
for dropout_probability in dropout_probabilities:
    for embedding_output_dim in embedding_output_dims:
        for layer_sizes in layer_sizes_list:
            for activation in activations:
                models_params_list.append(dict(
                    dropout_probability=dropout_probability,  
                    embedding_output_dim=embedding_output_dim,
                    layer_sizes=layer_sizes,
                    activation=activation))

print("%d models" % len(models_params_list))
models_params_explored = set.union(*[set(x) for x in models_params_list])
models_params_explored


48 models


{'activation', 'dropout_probability', 'embedding_output_dim', 'layer_sizes'}

In [ ]:
import sys


cv_df = defaultdict(list)
start = time.time()

for (allele, data) in train_data.items():
    print("Allele: %s" % allele)
    # data_df = allele_data_to_df(data)
    X, Y, weights, original_peptides = get_shuffled_fields(data)
            
    cv = sklearn.cross_validation.LabelKFold(original_peptides, n_folds = 3)
    for (fold_num, (train_indices, test_indices)) in enumerate(cv):
        print("-- fold #%d/3" % (fold_num + 1,))
        X_cv_train = X[train_indices]
        X_cv_test = X[test_indices]
        
        Y_cv_train = Y[train_indices]
        Y_cv_test = Y[test_indices]
        
        weights_cv_train = weights[train_indices]
        weights_cv_test = weights[test_indices]
        
        original_peptides_train = original_peptides[train_indices]
        original_peptides_test = original_peptides[test_indices]
        
        for (i, model_params) in enumerate(models_params_list):
            print("%10s fold %3d [%3d / %3d] train_size=%d test_size=%d impute=%s model=%s" %
                  (allele, fold_num, i, len(models_params_list), len(train_indices), len(test_indices), impute, model_params))
            sys.stdout.flush()
            predictor = mhcflurry.Class1BindingPredictor.from_hyperparameters(
                max_ic50=max_ic50,
                **model_params)

            fit_time = -time.time()
            predictor.fit(
                X_cv_train,
                Y_cv_train,
                sample_weights=weights_cv_train,
                verbose=False,
                n_training_epochs=200)
            fit_time += time.time()
    
            Y_cv_train_9mer_predictions = predictor.predict(X_cv_train)
            Y_cv_test_9mer_predictions = predictor.predict(X_cv_test)
            
            Y_train_true, Y_train_pred = collapse_9mer_affinities(
                Y_9mer_true=Y_cv_train,
                Y_9mer_pred=Y_cv_train_9mer_predictions,
                original_peptides=original_peptides_train)
            
            Y_test_true, Y_test_pred = collapse_9mer_affinities(
                Y_9mer_true=Y_cv_test,
                Y_9mer_pred=Y_cv_test_9mer_predictions,
                original_peptides=original_peptides_test)
            
            cv_df["allele"].append(allele)
            cv_df["allele_size"].append(len(Y))
            cv_df["train_size"].append(len(Y_cv_train))
            cv_df["model_params"].append(model_params)
            cv_df["fit_time"].append(fit_time)

            for (param, param_value) in model_params.items():
                cv_df[param].append(param_value)
            
            for (key, value) in make_scores(Y_train_true, Y_train_pred).items():
                cv_df["train_%s" % key].append(value)
                print("train %s: %f" % (key, value))
            
            for (key, value) in make_scores(
                    Y_test_true, 
                    Y_test_pred).items():
                cv_df["test_%s" % key].append(value)
                print("test %s: %f" % (key, value))


cv_df = pandas.DataFrame(cv_df)
cv_df["layer0_size"] = [x[0] for x in cv_df.layer_sizes]
print(time.time() - start)
cv_df

Allele: HLA-A0201
-- fold #1/3
 HLA-A0201 fold   0 [  0 /  48] train_size=21917 test_size=10959 impute=False model={'activation': 'tanh', 'embedding_output_dim': 16, 'dropout_probability': 0.0, 'layer_sizes': [16]}


In [ ]:
train_data["HLA-A0201"].X_index.shape

In [ ]:
cv_df = pandas.DataFrame(cv_df)
cv_df["layer0_size"] = [x[0] for x in cv_df.layer_sizes]
cv_df

In [ ]:
cv_df.to_csv("cv4.csv")

In [ ]:
group_columns = ["allele", "allele_size", "impute"]
group_columns.extend(models_params_explored)
group_columns.append("layer0_size")
group_columns.remove("layer_sizes")
print(mean_with_std(cv_df.groupby(group_columns).test_auc)) #.sort(inplace=False, ascending=False)



In [ ]:
def best_by(score):
    means = cv_df.groupby(group_columns)[score].mean().reset_index()
    max_rows = []
    for allele in means.allele.unique():
        max_rows.append(means.ix[means.allele == allele][score].argmax())
    return means.ix[max_rows]

In [ ]:
best_by('test_auc')


In [ ]:
best_by('test_tau')

In [ ]:
best_by('test_f1')